In [2]:
import numpy as np
np.int = int
np.float = float
np.complex = complex

import pypulseq as pp

# dummy system
system = pp.Opts(
    max_grad=28, grad_unit='mT/m',
    max_slew=150, slew_unit='T/m/s',
    rf_ringdown_time=20e-6, rf_dead_time=100e-6,
    adc_dead_time=20e-6,
    grad_raster_time=50e-6
)

c:\Users\endresjn\Documents\MRzero\MRzero-Core\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# *** template_A.ipynb -> FLASH.ipynb

# !!! CONFIG
# Field of view, imaging volume and resolution
fov = 200e-3
slice_thickness = 8e-3
Nread = 64
Nphase = 64

# Sequence's name
experiment_id = 'flash'

# !!! SEQUENCE DEFINITION
def flash_2D(fov=200e-3, slice_thickness=8e-3,
             n_read=64, n_phase=64,
             system=system,
             flip_angle=10, phase_cycling=84):
    """Linear, cartesian 2D FLASH with TR = 26 ms + 50 us * n_phase"""
    rf, gz, gzr = pp.make_sinc_pulse(
        flip_angle=flip_angle * np.pi / 180, duration=1e-3,
        slice_thickness=slice_thickness, apodization=0.5, time_bw_product=4,
        return_gz=True, system=system
    )

    adc_dur = n_phase * 50e-6
    gx = pp.make_trapezoid(channel='x', flat_area=n_read / fov, flat_time=adc_dur, system=system)
    adc = pp.make_adc(num_samples=n_read, duration=adc_dur, delay=gx.rise_time, system=system)
    gx_pre = pp.make_trapezoid(channel='x', area=-0.5 * gx.area, duration=5e-3, system=system)
    gx_spoil = pp.make_trapezoid(channel='x', area=1.5 * gx.area, duration=2e-3, system=system)

    seq = pp.Sequence(system)
    for i in range(n_phase):
        phase = 0.5 * phase_cycling * (2 + i + i**2)
        rf.phase_offset = (phase % 360) * np.pi / 180
        adc.phase_offset = rf.phase_offset

        phenc = (i - n_phase // 2) / fov

        seq.add_block(rf, gz)
        seq.add_block(gzr, pp.make_delay(5e-3))
        gp = pp.make_trapezoid(channel='y', area=phenc, duration=5e-3, system=system)
        seq.add_block(gx_pre, gp)
        seq.add_block(adc, gx)
        gp = pp.make_trapezoid(channel='y', area=-phenc, duration=5e-3, system=system)
        seq.add_block(gx_spoil, gp)
        seq.add_block(pp.make_delay(10e-3))

    seq.set_definition('FOV', [fov, fov, slice_thickness])
    return seq

# !!! SEQUENCE CREATION
seq = flash_2D(fov, slice_thickness, Nread, Nphase, system)
seq.set_definition('Name', experiment_id)
seq.write(experiment_id + '.seq')
